# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/smotghare/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/smotghare/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'f5dd61'. Skipping!
Property 'summary' already exists in node '212aee'. Skipping!
Property 'summary' already exists in node '924e36'. Skipping!
Property 'summary' already exists in node '890070'. Skipping!
Property 'summary' already exists in node '526243'. Skipping!
Property 'summary' already exists in node 'd07b5c'. Skipping!
Property 'summary' already exists in node 'b31fb9'. Skipping!
Property 'summary' already exists in node '21894d'. Skipping!
Property 'summary' already exists in node 'dd7b80'. Skipping!
Property 'summary' already exists in node 'c36bf8'. Skipping!
Property 'summary' already exists in node 'e8e208'. Skipping!
Property 'summary' already exists in node '3e45d4'. Skipping!
Property 'summary' already exists in node '95eb7e'. Skipping!
Property 'summary' already exists in node '70337f'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/41 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '212aee'. Skipping!
Property 'summary_embedding' already exists in node 'f5dd61'. Skipping!
Property 'summary_embedding' already exists in node '95eb7e'. Skipping!
Property 'summary_embedding' already exists in node 'b31fb9'. Skipping!
Property 'summary_embedding' already exists in node 'd07b5c'. Skipping!
Property 'summary_embedding' already exists in node '21894d'. Skipping!
Property 'summary_embedding' already exists in node '924e36'. Skipping!
Property 'summary_embedding' already exists in node '890070'. Skipping!
Property 'summary_embedding' already exists in node '526243'. Skipping!
Property 'summary_embedding' already exists in node 'dd7b80'. Skipping!
Property 'summary_embedding' already exists in node 'e8e208'. Skipping!
Property 'summary_embedding' already exists in node 'c36bf8'. Skipping!
Property 'summary_embedding' already exists in node '3e45d4'. Skipping!
Property 'summary_embedding' already exists in node '70337f'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 39, relationships: 476)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 39, relationships: 476)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### ✅ Answer:

Here are the three types of query synthesizers and what they do:

1. **SingleHopSpecificQuerySynthesizer (50%)**:
    - Purpose : Creates simple, direct questions
    - Example : "What is the interest rate for Direct Subsidized Loans?"
    - Graph Usage : Queries single nodes or direct node relationships
    - Complexity : Low - can be answered from one document section
2. **MultiHopAbstractQuerySynthesizer (25%)**:
    - Purpose : Creates conceptual questions requiring synthesis
    - Example : "How does the FAFSA application process impact loan eligibility?"
    - Graph Usage : Traverses multiple nodes and relationship types
    - Complexity : High - requires understanding connections between concepts
3. **MultiHopSpecificQuerySynthesizer (25%)**:
    - Purpose : Creates specific questions requiring multiple facts
    - Example : "What documents are needed for FAFSA verification and how does this relate to loan disbursement timelines?"
    - Graph Usage : Follows specific paths through multiple connected nodes
    - Complexity : Medium-High - requires combining specific facts from multiple sources

The distribution (50%, 25%, 25%) ensures a good mix of simple and complex questions, mimicking real-world user behavior where most questions are straightforward, but some require deeper understanding.    


Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What information is available about Chapter 2 ...,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not include specific...,single_hop_specifc_query_synthesizer
1,Can you tell me about Chapter 3 and how it rel...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
2,How does Title IV regulation relate to payment...,[Non-Term Characteristics A program that measu...,"According to the context, the payment period i...",single_hop_specifc_query_synthesizer
3,What is TEACH Grant?,[both the credit or clock hours and the weeks ...,The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
4,What is the significance of Title IV in relati...,[Disbursement Timing in Subscription-Based Pro...,Disbursement timing in subscription-based prog...,single_hop_specifc_query_synthesizer
5,Based on the guidelines and exceptions for cli...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Measuring progress in clock hours causes a pro...,multi_hop_abstract_query_synthesizer
6,How do the timing and scheduling constraints o...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
7,How do the timing and scheduling constraints o...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The timing and scheduling constraints of clini...,multi_hop_abstract_query_synthesizer
8,How does Volume 8 explain the impact of accele...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 8 details that accelerated progression ...,multi_hop_specific_query_synthesizer
9,"In Appendix A and B, how do disbursement rules...",[<1-hop>\n\nboth the credit or clock hours and...,"According to the context, Appendix A discusses...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '96d97d'. Skipping!
Property 'summary' already exists in node 'acf826'. Skipping!
Property 'summary' already exists in node 'ac8ac5'. Skipping!
Property 'summary' already exists in node 'f0be18'. Skipping!
Property 'summary' already exists in node '094d74'. Skipping!
Property 'summary' already exists in node '103bb2'. Skipping!
Property 'summary' already exists in node '9629b9'. Skipping!
Property 'summary' already exists in node 'ac31ba'. Skipping!
Property 'summary' already exists in node '18f731'. Skipping!
Property 'summary' already exists in node 'c3c282'. Skipping!
Property 'summary' already exists in node '561809'. Skipping!
Property 'summary' already exists in node 'ad841c'. Skipping!
Property 'summary' already exists in node 'b4cbdf'. Skipping!
Property 'summary' already exists in node '67b6a9'. Skipping!


...

Property 'summary_embedding' already exists in node '96d97d'. Skipping!
Property 'summary_embedding' already exists in node 'acf826'. Skipping!
Property 'summary_embedding' already exists in node 'ac8ac5'. Skipping!
Property 'summary_embedding' already exists in node '561809'. Skipping!
Property 'summary_embedding' already exists in node 'c3c282'. Skipping!
Property 'summary_embedding' already exists in node 'f0be18'. Skipping!
Property 'summary_embedding' already exists in node 'ac31ba'. Skipping!
Property 'summary_embedding' already exists in node '67b6a9'. Skipping!
Property 'summary_embedding' already exists in node 'ad841c'. Skipping!
Property 'summary_embedding' already exists in node '103bb2'. Skipping!
Property 'summary_embedding' already exists in node 'b4cbdf'. Skipping!
Property 'summary_embedding' already exists in node '094d74'. Skipping!
Property 'summary_embedding' already exists in node '9629b9'. Skipping!
Property 'summary_embedding' already exists in node '18f731'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Volume 2 in the context of academic ye...,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not specify what Vol...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about in relation to a...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What information does Volume 8 provide regardi...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 discusses the inclusion of clinical w...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in federal a...,[Non-Term Characteristics A program that measu...,Non-Term Characteristics refer to programs tha...,single_hop_specifc_query_synthesizer
4,How does the timing of disbursement requiremen...,[<1-hop>\n\nboth the credit or clock hours and...,The disbursement timing for federal aid varies...,multi_hop_abstract_query_synthesizer
5,how disbursement timing in subscription-based ...,[<1-hop>\n\nboth the credit or clock hours and...,In clock-hour or non-term credit-hour programs...,multi_hop_abstract_query_synthesizer
6,how impact of accelerated progression affect d...,[<1-hop>\n\nboth the credit or clock hours and...,The impact of accelerated progression in clock...,multi_hop_abstract_query_synthesizer
7,How does disbursement timing in subscription-b...,[<1-hop>\n\nboth the credit or clock hours and...,"In clock-hour programs, disbursement timing de...",multi_hop_abstract_query_synthesizer
8,Waht chapters cover the disbursement timing in...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Chapter 2 discusses the disbursement timing in...,multi_hop_specific_query_synthesizer
9,Volume 2 and Volume 8 what do they say about d...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","In Volume 2, it talks about academic years, we...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [27]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [29]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [32]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans)  \n- Subsidized and Unsubsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010)  \n- Federal SLS Loans (made under the FFEL Program before July 1, 2010)  \n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)  \n\nNote: No new FFEL Program loans have been made since July 1, 2010. Graduate or professional students are eligible only for Direct Unsubsidized Loans and Direct PLUS Loans, not Direct Subsidized Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [33]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:


#### ✅ Answer:

**QA Evaluator**:
- Purpose: Measures factual correctness
- Method: Compares generated answer against reference answer
- Output: Binary score (1 = correct, 0 = incorrect)

**Labeled Helpfulness Evaluator**:
- Purpose: Evaluates response utility beyond just correctness
- Method: LLM judges helpfulness considering the reference answer
- Output: Binary score (1 = helpful, 0 = not helpful)

**Empathy Evaluator**:
- Purpose: Measures user experience and tone
- Method: LLM evaluates emotional intelligence of response
- Output: Binary score (1 = empathetic, 0 = not empathetic)

This multi-dimensional evaluation approach captures different aspects of RAG system quality:

1. Correctness (qa_evaluator) → "Is the answer right?"
2. Utility (helpfulness_evaluator) → "Does this actually help the user?"
3. Experience (empathy_evaluator) → "Does this make the user feel good about the interaction?"

This combination ensures that a RAG system doesn't just provide accurate information, but does so in a way that's genuinely useful and creates a positive user experience.

## LangSmith Evaluation

In [35]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'cold-copy-6' at:
https://smith.langchain.com/o/7d3c7038-0aea-414a-89bb-83aec330bb8e/datasets/e5711008-0937-417a-bf34-2143733f6371/compare?selectedSessions=735f5a42-053e-49eb-947b-f0764a2810ee




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 8 clinical work include in...,I don't know.,None,Volume 8 talks about clinical work in standard...,0,0,0,0.923674,1370c016-7028-469a-80bb-20f3cbfd8dfb,31671581-5f85-4bf8-9616-872711816fd4
1,Whre do I find info in Appendix A and B about ...,"Based on the provided context, information abo...",None,"The detailed guidance on disbursment rules, in...",1,0,0,3.865968,9f0e2927-56da-4d62-a7fb-0f3c45d8142a,e0005e60-9d35-4931-8b29-899833f72a24
2,Volume 2 and Volume 8 what do they say about d...,I don't know.,None,"In Volume 2, it talks about academic years, we...",0,0,0,0.910634,3709b88b-c763-4d7a-abb2-fe36006416a3,fbb3ec4d-22c7-417b-b12c-1109e6e8014b
3,Waht chapters cover the disbursement timing in...,"The context indicates that Chapter 1 covers ""A...",None,Chapter 2 discusses the disbursement timing in...,0,0,0,3.939704,ba20f5a8-8d20-44b3-a057-74792e3449a9,4f7c126d-7dd3-4d0f-9705-ef97c60b2bf6
4,How does disbursement timing in subscription-b...,Disbursement timing in subscription-based prog...,None,"In clock-hour programs, disbursement timing de...",1,0,0,9.323563,88ce01f0-3b4b-4472-9ccc-729c984cd61e,8d170c7f-52cf-4042-ae37-c627b65d94cc
5,how impact of accelerated progression affect d...,Based on the provided context:\n\n**Impact of ...,None,The impact of accelerated progression in clock...,0,0,0,7.838984,00d3cf18-85c0-406c-a1c2-ee2197b348b0,ec915b23-d06d-47ad-bf3a-0d668cb42937
6,how disbursement timing in subscription-based ...,Based on the provided context:\n\n1. **Disburs...,None,In clock-hour or non-term credit-hour programs...,1,1,0,9.364849,f1734fc6-5a48-4abb-87a3-cfb40b7f3d37,9b32473d-dc38-4b28-ba58-bf164d6f796d
7,How does the timing of disbursement requiremen...,"Based on the provided context, the timing of d...",None,The disbursement timing for federal aid varies...,1,1,0,10.064180,b24a264f-e254-4690-a700-cd885c77a3ae,ee626af6-a8a0-47fb-8f36-3513acd8af14
8,What are Non-Term Characteristics in federal a...,Non-Term Characteristics in federal aid progra...,None,Non-Term Characteristics refer to programs tha...,1,1,0,2.507893,12c528f0-c131-4c48-b3f6-35cef84bc08a,eeb12a3f-ca56-41fd-b650-9d0365a69907
9,What information does Volume 8 provide regardi...,"Volume 8, Chapter 3 provides additional guidan...",None,Volume 8 discusses the inclusion of clinical w...,1,0,0,1.639437,81185286-a393-48ea-acca-d9bd4c7b7210,7b3dd01c-4035-4196-998e-0d64b5f580c3


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [36]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [37]:
rag_documents = docs

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer:

Larger chunk size (1000 vs 500 characters) impacts performance because:

**Potential Improvements:**
- More context per retrieval: Each chunk contains more complete information, helping answer complex questions
- Better context continuity: Related information is less likely to be split across separate chunks
- Comprehensive answers: Users get more complete responses since each retrieved chunk covers more ground

**Potential Trade-offs:**
- Less precision: Larger chunks may include irrelevant information that dilutes the signal
- Retrieval noise: Important details might get buried in larger text blocks
- Context window usage: Uses more of the LLM's context window per chunk

**Impact on Metrics:**
- Correctness: May improve for questions needing broader context
- Helpfulness: Likely improves due to more comprehensive answers  
- Empathy: Minimal direct impact

**The key insight**: Chunk size is about balancing **precision** (smaller chunks = more focused) vs **completeness** (larger chunks = more context). The optimal size depends on the specific use case and question complexity.


In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer:

Upgrading from `text-embedding-3-small` to `text-embedding-3-large` impacts performance because:

**Key Improvements:**
- Higher dimensional vectors: More nuanced semantic representations (1536 → 3072 dimensions)
- Better semantic understanding: More accurate capture of meaning and context
- Improved similarity matching: Better at finding truly relevant chunks for queries
- Enhanced retrieval precision: Reduces false positives in document retrieval

**Impact on RAG Pipeline:**
- Better context retrieval: More relevant chunks = more accurate answers
- Reduced noise: Less irrelevant information passed to the LLM
- Complex query handling: Better at understanding nuanced or abstract questions

**Expected Metric Changes:**
- Correctness: Should improve due to more relevant context retrieval
- Helpfulness: Likely improves from better-targeted information
- Empathy: Minimal direct impact

**Trade-offs:**
- Higher costs: Larger model = more expensive API calls
- Slightly slower: More computation for embeddings

**The key insight**: Better embeddings = better retrieval = better RAG performance, since the quality of retrieved context directly impacts the LLM's ability to provide accurate, helpful responses.


In [40]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [41]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [42]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [43]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Thank you for your question. Based on the information provided in the context, there are several kinds of loans available to students and parents to help with college costs:\n\n1. **Direct Subsidized Loans** – These are loans where the amount you can borrow depends on your financial need. Interest is paid by the government while you’re in school at least half-time.\n\n2. **Direct Unsubsidized Loans** – These loans are available regardless of financial need. Interest accrues while you’re in school and you’re responsible for paying it.\n\n3. **Direct PLUS Loans** – These loans can be taken out by the parent of a dependent student to cover the cost of attendance (COA), assuming eligibility requirements are met. Graduate or professional students can also apply for PLUS loans. There’s no fixed loan limit for PLUS loans, but they cannot exceed the student’s COA minus other financial aid.\n\n4. Additionally, if a parent cannot obtain a Direct PLUS Loan, dependent students may be eligible for

Finally, we can evaluate the new chain on the same test set!

In [44]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'aching-grip-12' at:
https://smith.langchain.com/o/7d3c7038-0aea-414a-89bb-83aec330bb8e/datasets/e5711008-0937-417a-bf34-2143733f6371/compare?selectedSessions=c388feb4-bcc6-463a-abfc-5ed306b49642




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 8 clinical work include in...,Thank you for your thoughtful question about h...,None,Volume 8 talks about clinical work in standard...,1,1,1,4.759178,1370c016-7028-469a-80bb-20f3cbfd8dfb,8c5d9446-3313-4f27-907f-18c8a2e630ae
1,Whre do I find info in Appendix A and B about ...,Thank you for your question! It sounds like yo...,None,"The detailed guidance on disbursment rules, in...",1,1,1,4.426869,9f0e2927-56da-4d62-a7fb-0f3c45d8142a,92395e7c-16f9-4533-880f-b51c74058364
2,Volume 2 and Volume 8 what do they say about d...,Thank you for your thoughtful question. Based ...,None,"In Volume 2, it talks about academic years, we...",0,0,1,6.333469,3709b88b-c763-4d7a-abb2-fe36006416a3,763981df-4ec4-45ac-9ec5-96a92453a099
3,Waht chapters cover the disbursement timing in...,Thank you for your thoughtful question. Based ...,None,Chapter 2 discusses the disbursement timing in...,1,0,1,5.302480,ba20f5a8-8d20-44b3-a057-74792e3449a9,894f4567-2407-47d8-838c-721f35877f6f
4,How does disbursement timing in subscription-b...,Thank you for your thoughtful question. It sho...,None,"In clock-hour programs, disbursement timing de...",1,0,1,6.436059,88ce01f0-3b4b-4472-9ccc-729c984cd61e,41fd79fe-3a4d-4c10-af05-fb0f28fb91a3
5,how impact of accelerated progression affect d...,Thank you for your thoughtful question. I unde...,None,The impact of accelerated progression in clock...,1,0,1,6.464208,00d3cf18-85c0-406c-a1c2-ee2197b348b0,85c269da-37fc-40b5-ad4e-5c1c30a05fc7
6,how disbursement timing in subscription-based ...,Thank you for your thoughtful question. I unde...,None,In clock-hour or non-term credit-hour programs...,1,0,1,5.788389,f1734fc6-5a48-4abb-87a3-cfb40b7f3d37,bfc52236-6b16-423e-be09-a31f006c3390
7,How does the timing of disbursement requiremen...,Thank you for your thoughtful question. It’s c...,None,The disbursement timing for federal aid varies...,1,1,1,7.422411,b24a264f-e254-4690-a700-cd885c77a3ae,c7170128-3cac-4dd6-ace2-94e86f58d6f9
8,What are Non-Term Characteristics in federal a...,Thank you for your question—it's absolutely im...,None,Non-Term Characteristics refer to programs tha...,1,1,1,4.475011,12c528f0-c131-4c48-b3f6-35cef84bc08a,25996104-7350-4f2b-abdc-ed1a802a8844
9,What information does Volume 8 provide regardi...,Thank you for your thoughtful question. Based ...,None,Volume 8 discusses the inclusion of clinical w...,0,0,1,2.296580,81185286-a393-48ea-acca-d9bd4c7b7210,16e4b4f1-f345-4ddb-a9dd-a06f511a2e7c


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### ✅ Answer: 

**Configuration Differences:**

- Chain 1 (`cold-copy-6`): 500-character chunks + `text-embedding-3-small`
- Chain 2 (`aching-grip-12`): 1000-character chunks + `text-embedding-3-large`

-----------
-----------

![Loan Dataset Comparison Outputs](https://i.imgur.com/QLqeqSw.png)

-----------
-----------

![Loan Dataset Comparison Chart](https://i.imgur.com/73wsKEP.png)

-----------
-----------

**Performance Results from the Charts:**

**1.Correctness Metric:**

- `cold-copy-6`: ~0.58 (lower performance)
- `aching-grip-12`: ~0.83 (significantly higher performance)
- Improvement: ~43% increase

**2.Empathy Metric:**

- `cold-copy-6`: ~0.58 (lower performance)
- `aching-grip-12`: ~1.0 (perfect score)
- Improvement: ~72% increase

**3.Helpfulness Metric:**

- `cold-copy-6`: ~0.25 (low performance)
- `aching-grip-12`: ~0.42 (moderate improvement)
- Improvement: ~68% increase

**Observations:**

**✅ Correctness Improved Significantly**

- Larger chunks (1000 vs 500): More complete context per retrieval, reducing information fragmentation
- Better embedding model (3-large vs 3-small): More nuanced semantic understanding leads to better relevant document retrieval
- Combined effect: More accurate answers due to both better context and better retrieval precision

**🤝 Empathy Reached Perfect Score**

- Richer context: Larger chunks provide more complete information about user concerns and emotional context
- Better semantic matching: The larger embedding model better captures emotional nuances in queries
- Complete responses: Less fragmented information allows for more empathetic, comprehensive responses

**💡 Helpfulness Showed Moderate Improvement**

- More comprehensive answers: Larger chunks provide more complete solutions
- Better context retrieval: Improved embedding model finds more relevant helpful information
- Still room for improvement: This metric may require additional optimization beyond just chunking/embeddings

**🔍 Key Insight:**

The combination of **larger chunk sizes + more sophisticated embeddings** created a synergistic effect where better retrieval led to more contextually rich, accurate, and empathetic responses, with the most dramatic improvements in correctness and empathy metrics.